In [1]:
import pandas as pd
import pandasql as ps

In [16]:
ifood = pd.read_csv("ifood-restaurants-february-2021.csv")

ps.sqldf("SELECT delivery_time FROM ifood limit 5")

,delivery_time
0,27
1,61
2,70
3,63
4,58


In [15]:
# Tipo de dados
for i in ifood.columns:
    display(ps.sqldf(f"SELECT DISTINCT typeof({i}) FROM ifood"))

,typeof(availableForScheduling)
0,integer


,typeof(avatar)
0,text
1,null


,typeof(category)
0,text


,typeof(delivery_fee)
0,real


,typeof(delivery_time)
0,integer


...

,typeof(paymentCodes)
0,text
1,null


,typeof(price_range)
0,text


,typeof(rating)
0,real


,typeof(tags)
0,text


,typeof(url)
0,text


In [13]:
# Contagem de nulos/não nulos
ps.sqldf( """
                    SELECT 
                        SUM(CASE WHEN delivery_fee IS NULL THEN 1 ELSE 0 END) as countar_nulls
                        , SUM(CASE WHEN delivery_fee IS NULL THEN 0 ELSE 1 END) as countar__n_nulls
                    FROM ifood
               ;"""

)

,countar_nulls,countar__n_nulls
0,0,406399
